In [ ]:
# import necessary libraries

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve

import matplotlib.pyplot as plt

import time

print(tf.__version__)

In [ ]:
# read in the train and test data for a certain lesion type

# Basepath depends on the lesion
# LesionType = 'SoftExudates'
LesionType = 'HardExudates'
# LesionType = 'Microaneurysms'
# LesionType = 'Hemorrhages'

In [ ]:
# Basepath for Jupyter notebooks:
Basepath = 'C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/Kaggle/Arrays_10GB/'

# test data
test_images = np.float32(np.load(Basepath + 'test_images_Final.npy'))
print('Shape test images: {}'.format(test_images.shape))

In [ ]:
# path where the model was stored

# basepath for jupyter notebooks
base_path = 'C:/Users/lunam/Documents/1steMaster/Stage/Code_Final/DR_classification/FeatureBasedClassification/UNet_Softmax_GPU/Logs/' + LesionType + '/'

# direction where the trained models are stored
log_dir_model = base_path + 'Trained_Model/'

In [ ]:
# The UNet network
def UNet(drop_prob = 0.1, init_filters = 64):
    '''This function defines the original UNet network'''
  
    # initialization of the weights
    W_init = tf.initializers.GlorotUniform()
    
    # Unet network
            
    # LEFT part
    # input layer
    # input_image = tf.keras.layers.InputLayer(input_shape = (512,512,3))
    input_image = tf.keras.layers.Input(shape = (512,512,3))
        
    # Convolutional block 1
    conv2d_1 = tf.keras.layers.Conv2D(init_filters, (3, 3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(input_image)
    conv2d_2 = tf.keras.layers.Conv2D(init_filters, (3, 3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_1)
    pool_1 = tf.keras.layers.MaxPool2D((2, 2), (2, 2))(conv2d_2)
    dropout_1 = tf.keras.layers.Dropout(rate = drop_prob)(pool_1)
            
    # Convolutional block 2
    conv2d_3 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_1)
    conv2d_4 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_3)
    pool_2 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_4)
    dropout_2 = tf.keras.layers.Dropout(rate = drop_prob)(pool_2 )
            
    # Convolutional block 3
    conv2d_5 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_2)
    conv2d_6 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_5)
    pool_3 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_6)
    dropout_3 = tf.keras.layers.Dropout(rate = drop_prob)(pool_3)
            
    # Convolutional block 4
    conv2d_7 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_3)
    conv2d_8 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_7)
    pool_4 = tf.keras.layers.MaxPool2D((2,2), (2, 2))(conv2d_8)
    dropout_4 = tf.keras.layers.Dropout(rate = drop_prob)(pool_4)
            
    # MIDDLE part
    conv2d_9 = tf.keras.layers.Conv2D(16*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_4)
    conv2d_10 = tf.keras.layers.Conv2D(16*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_9)
            
            
    # RIGHT part
    # Convolutional block 1
    upsampling_1 = tf.keras.layers.UpSampling2D((2,2))(conv2d_10)
    concat_1 = tf.keras.layers.Concatenate(3)([upsampling_1, conv2d_8])
    dropout_5 = tf.keras.layers.Dropout(rate = drop_prob)(concat_1)
    conv2d_11 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_5)
    conv2d_12 = tf.keras.layers.Conv2D(8*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_11)
            
    # Convolutional block 2
    upsampling_2 = tf.keras.layers.UpSampling2D((2,2))(conv2d_12)
    concat_2 = tf.keras.layers.Concatenate(3)([upsampling_2, conv2d_6])
    dropout_6 = tf.keras.layers.Dropout(rate = drop_prob)(concat_2)
    conv2d_13 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_6)
    conv2d_14 = tf.keras.layers.Conv2D(4*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_13)
        
            
    # Convolutional block 3
    upsampling_3 = tf.keras.layers.UpSampling2D((2,2))(conv2d_14)
    concat_3 = tf.keras.layers.Concatenate(3)([upsampling_3,conv2d_4])
    dropout_7 = tf.keras.layers.Dropout(rate = drop_prob)(concat_3)
    conv2d_15 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_7)
    conv2d_16 = tf.keras.layers.Conv2D(2*init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_15)
            
    # Convolutional block 4
    upsampling_4 = tf.keras.layers.UpSampling2D((2,2))(conv2d_16)
    concat_4 = tf.keras.layers.Concatenate(3)([upsampling_4,conv2d_2])
    dropout_8 = tf.keras.layers.Dropout(rate = drop_prob)(concat_4)
    conv2d_17 = tf.keras.layers.Conv2D(init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(dropout_8)
    conv2d_18 = tf.keras.layers.Conv2D(init_filters, (3,3), activation= tf.nn.relu, kernel_initializer= W_init, padding = 'same')(conv2d_17)
            
    # ouput layer
    output_image = tf.keras.layers.Conv2D(2, (1,1), kernel_initializer= W_init, padding = 'same')(conv2d_18)

    # define the model
    model = tf.keras.Model(inputs=input_image, outputs=output_image)
    return model

In [ ]:
def Reload_and_Predict(TestImages, Drop_Prob = 0.1, Init_Filters = 64, log_dir_trained_model = log_dir_model, test_batch = 6, epochs = 10):
    '''This function reloads a trained model and predicts the annotations of images'''

    # Define the model for testing
    print('Define the model for testing')
    model = UNet(drop_prob = Drop_Prob, init_filters = Init_Filters)

    # Load the stored weights of the trained model
    print('Restoring weight parameters...')
    model.load_weights(log_dir_trained_model + 'UNet_Softmax_'+ str(epochs) +'_epochs')
    print('Model restored...')

    # predict the annotations for the test images
    print('Predicting the test cases...')

    predicted_prob_maps = []
    n_batches = int(TestImages.shape[0] / test_batch)
    if TestImages.shape[0] % test_batch != 0:
        print('Amount of test images is not dividable by the chosen test batch size!')
        print('Number of images: {}'.format(TestImages.shape[0]))
        print('Chosen batch size: {}'.format( test_batch))

    for i in range(n_batches):
        start = i*test_batch
        end = start + test_batch
        test_logits = model(TestImages[start:end], training = False)
        test_pos_prob = tf.nn.softmax(test_logits)[:,:,:,1]
        test_pos_prob = np.array(test_pos_prob)

        if i == 0:
            predicted_prob_maps = test_pos_prob
        else:
            predicted_prob_maps = np.vstack((predicted_prob_maps, test_pos_prob))
    print('Test data predicted')
  
    # test_positive_prob gives the output probability map that can be used as an input to the test function
    return predicted_prob_maps

In [ ]:
predicted_annot = Reload_and_Predict(test_images, Drop_Prob = 0.1, Init_Filters = 64, log_dir_trained_model = log_dir_model, epochs = 2, test_batch = 8)
# epochs are needed to define the name of the file to restore the model
# test batch has to be defined as a number through which the total amount of test images if dividable

In [ ]:
np.save('C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/Kaggle/Arrays_10GB_float32/test_annotations_'+LesionType+'.npy', predicted_annot)